In [ ]:
# Libs Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Libs SQL
import sqlite3 as sql

In [ ]:
# jojrojrojrojr

In [ ]:
df_fires_raw = pd.read_csv("/content/drive/MyDrive/Data/1_Projets/Feux de forets/Fires_Raw.csv")
df_nwcg_raw = pd.read_csv("/content/drive/MyDrive/Data/1_Projets/Feux de forets/NWCG.csv")

<ipython-input-8-62c27a21e796>:1: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,18,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fires_raw = pd.read_csv("/content/drive/MyDrive/Data/1_Projets/Feux de forets/Fires_Raw.csv")


In [ ]:
df_fires_raw.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63.0,63.0,Plumas,0001AD100000E864C2925F405EC0E0C86C98BA044440E8...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61.0,61.0,Placer,0001AD10000054B6EE6AE2195EC090C6555D6E77434054...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17.0,17.0,El Dorado,0001AD100000D0A5A057132F5EC050BB662CF97D4340D0...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3.0,3.0,Alpine,0001AD10000094ACA30D74FA5DC0E85400C69247434094...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3.0,3.0,Alpine,0001AD10000040E3AA2EB7FB5DC0E85400C69247434040...


In [ ]:
!pip install openmeteo-requests requests_cache retry-requests

In [7]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [8]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2024-01-06",
	"end_date": "2024-01-20",
	"hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 52.5483283996582°E 13.407821655273438°N
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                   date  temperature_2m
0   2024-01-06 00:00:00          1.0085
1   2024-01-06 01:00:00          0.0585
2   2024-01-06 02:00:00         -0.3415
3   2024-01-06 03:00:00         -0.4915
4   2024-01-06 04:00:00         -0.2915
..                  ...             ...
355 2024-01-20 19:00:00          0.1585
356 2024-01-20 20:00:00         -0.7915
357 2024-01-20 21:00:00         -1.5915
358 2024-01-20 22:00:00         -2.4415
359 2024-01-20 23:00:00         -3.2415

[360 rows x 2 columns]


In [1]:
import functions_framework

ModuleNotFoundError: No module named 'functions_framework'